In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/acoes/busca-avancada/')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[3]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[4]/div/div[1]/div[2]/a/span').click()
time.sleep(2)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'
dest = '/Users/scheiner/Documents/GitHub/StatusInvest/stocks.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [3]:
### Tratamento dos dados importados:

df = pd.read_csv("stocks.csv", sep=';', decimal=',', thousands='.', encoding='ISO-8859-1' )
    
df['P/L_IDEAL'] = df[' VPA'] / df[' LPA']
df['UP/DOWNSIDE'] = (df[' VPA'] / df['PRECO'])-1
    
valuation = df.loc[:, ['TICKER', 'PRECO', ' VPA' , 'DY', ' LPA', 'P/L', 'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 
                           'P/VP', 'EV/EBIT' ,'LIQ. CORRENTE', 'MARG. LIQUIDA', 'DIV. LIQ. / PATRI.']]

valuation.fillna(value=0, inplace=True)
valuation.rename(columns={'LIQ. CORRENTE': "LIQ.CORR" }, inplace=True)
valuation.rename(columns={'MARG. LIQUIDA': "MARG.LIQ" }, inplace=True)
valuation.rename(columns={'DIV. LIQ. / PATRI.': "DIV.LIQ/PL" }, inplace=True)
    
    
valuation = valuation.loc[valuation['DY'] >= 1].loc[
    valuation['P/L'] >= 0.01].loc[valuation['P/L'] < 15].loc[
    valuation['ROE'] >= 6].loc[
    valuation['EV/EBIT'] >= 0.01].loc[valuation['EV/EBIT'] <= 15].loc[
    valuation['LIQ.CORR'] >= 0.5]


In [4]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = valuation['TICKER']
lista_stocks = ticker.to_list()

lista_indicadores_stocks = []
for stocks in lista_stocks:

    #Criando url
    url = f"https://statusinvest.com.br/acoes/{stocks}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        PRECO = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[1]/div/div[1]/strong').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[1]/strong').text
        DY = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[4]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[2]/div/span[2]/b').text
        QTDEPAPEL = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div/div[2]/div[9]/div/div/strong').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div/div[3]/div/div[3]/div/div/div/a/strong').text
        
        

        #Criando dict
        dicionario = {  "stocks": stocks, 
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "QTDEPAPEL": QTDEPAPEL,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        lista_indicadores_stocks.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()


In [5]:
### Preparando os dados e Dataframe

list_stocks = pd.DataFrame.from_dict(lista_indicadores_stocks)

list_stocks = list_stocks.replace('-', '')
list_stocks = list_stocks.replace('-%', '')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.lstrip('R$ ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.lstrip('R$ ')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.strip('- ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.strip('- ')
list_stocks['VAL_12M'] = (list_stocks['VAL_12M']).str.rstrip('%')
list_stocks['VAL_MES'] = (list_stocks['VAL_MES']).str.rstrip('%')
list_stocks.to_csv('list_stocks.csv')

list_stocks = pd.read_csv("list_stocks.csv", sep=',', decimal=',', thousands='.', encoding='ISO-8859-1' )

list_stocks = list_stocks.drop(['Unnamed: 0'],axis=1)
list_stocks.fillna(value=0, inplace=True)

list_stocks = list_stocks.replace('AcessÃ³rios', 'Acessorios')
list_stocks = list_stocks.replace('Armas e MuniÃ§Ãµes', 'Armas e Municoes')
list_stocks = list_stocks.replace('Artefatos de Ferro e AÃ§o', 'Artefatos de Ferro e Aco')
list_stocks = list_stocks.replace('AutomÃ³veis e Motocicletas', 'Automoveis e Motocicletas')
list_stocks = list_stocks.replace('AÃ§ucar e Alcool', 'Acucar e Alcool')
list_stocks = list_stocks.replace('CalÃ§ados', 'Calcados')
list_stocks = list_stocks.replace('ConstruÃ§Ã£o Pesada', 'Construcao Pesada')
list_stocks = list_stocks.replace('EletrodomÃ©sticos', 'Eletrodomesticos')
list_stocks = list_stocks.replace('Energia ElÃ©trica', 'Energia Eletrica')
list_stocks = list_stocks.replace('Equipamentos e ServiÃ§os', 'Equipamentos e Servicos')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o de ImÃ³veis', 'Exploracao de Imoveis')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o de Rodovias', 'Exploracao de Rodovias')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o. Refino e DistribuiÃ§Ã£o', 'Exploracao Refino e Distribuicao')
list_stocks = list_stocks.replace('GestÃ£o de Recursos e Investimentos', 'Gestao Recursos e Investimentos')
list_stocks = list_stocks.replace('GÃ¡s', 'Gas')
list_stocks = list_stocks.replace('IncorporaÃ§Ãµes', 'Incorporacoes')
list_stocks = list_stocks.replace('IntermediaÃ§Ã£o ImobiliÃ¡ria', 'Intermediacao Imobiliaria')
list_stocks = list_stocks.replace('LogÃ­stica', 'Logistica')
list_stocks = list_stocks.replace('Material AeronÃ¡utico e de Defesa', 'Material Aeronautico e Defesa')
list_stocks = list_stocks.replace('Material RodoviÃ¡rio', 'Material Rodoviario')
list_stocks = list_stocks.replace('Minerais MetÃ¡licos', 'Minerais Metalicos')
list_stocks = list_stocks.replace('MÃ¡q. e Equip. ConstruÃ§Ã£o e AgrÃ­colas', 'Maq Equip Construcao e Agricolas')
list_stocks = list_stocks.replace('MÃ¡q. e Equip. Industriais', 'Maq Equip Industriais')
list_stocks = list_stocks.replace('MÃ³veis', 'Moveis')
list_stocks = list_stocks.replace('PetroquÃ­micos', 'Petroquimicos')
list_stocks = list_stocks.replace('Produtos para ConstruÃ§Ã£o', 'Produtos para Construcao')
list_stocks = list_stocks.replace('ProduÃ§Ã£o de Eventos e Shows', 'Producao de Eventos e Shows')
list_stocks = list_stocks.replace('ProduÃ§Ã£o e DifusÃ£o de Filmes e Programas', 'Producao Difusao de Filmes e Programas')
list_stocks = list_stocks.replace('Programas de FidelizaÃ§Ã£o', 'Programas de Fidelizacao')
list_stocks = list_stocks.replace('Programas e ServiÃ§os', 'Programas e Servicos')
list_stocks = list_stocks.replace('QuÃ­micos Diversos', 'Quimicos Diversos')
list_stocks = list_stocks.replace('Serv.MÃ©d.Hospit..AnÃ¡lises e DiagnÃ³sticos', 'Serv Mao Hosp Analises e Diagnosticos')
list_stocks = list_stocks.replace('ServiÃ§os Diversos', 'Servicos Diversos')
list_stocks = list_stocks.replace('ServiÃ§os Educacionais', 'Servicos Educacionais')
list_stocks = list_stocks.replace('ServiÃ§os Financeiros Diversos', 'Servicos Financeiros Diversos')
list_stocks = list_stocks.replace('Soc. CrÃ©dito e Financiamento', 'Soc Credito e Financiamento')
list_stocks = list_stocks.replace('Tecidos. VestuÃ¡rio e CalÃ§ados', 'Tecidos Vestuario e Calcados')
list_stocks = list_stocks.replace('TelecomunicaÃ§Ãµes', 'Telecomunicacoes')
list_stocks = list_stocks.replace('Transporte AÃ©reo', 'Transporte Aereo')
list_stocks = list_stocks.replace('Transporte FerroviÃ¡rio', 'Transporte Ferroviario')
list_stocks = list_stocks.replace('Transporte HidroviÃ¡rio', 'Transporte Hidroviario')
list_stocks = list_stocks.replace('UtensÃ­lios DomÃ©sticos', 'Utensilios Domesticos')
list_stocks = list_stocks.replace('VestuÃ¡rio', 'Vestuario')
list_stocks = list_stocks.replace('Ãgua e Saneamento', 'Agua e Saneamento')

valuation.rename(columns={'TICKER': "TICKERS" }, inplace=True)

list_stocks.rename(columns={'stocks': "TICKERS" }, inplace=True)

investing = pd.merge(valuation, list_stocks, on='TICKERS')
#investing.sort_index(axis=1, inplace=True)


In [6]:
investing

,TICKERS,PRECO,VPA,DY,LPA,P/L,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,...,MARG.LIQ,DIV.LIQ/PL,MIN_MES,MAX_MES,MIN_52S,MAX_52S,VAL_12M,VAL_MES,QTDEPAPEL,SEGMENTO
0,ABCB4,19.40,23.98,8.43,3.48,5.35,6.890805,0.236082,14.50,0.78,...,19.35,0.00,18.35,19.40,15.27,21.24,3.58,3.91,226090118,Bancos
1,AFLT3,8.60,4.30,2.95,0.62,14.33,6.935484,-0.500000,14.46,2.07,...,48.96,-0.07,8.60,9.01,8.07,10.50,-6.01,-3.37,63084700,Energia Eletrica
2,AGRO3,27.56,21.46,12.49,2.62,9.88,8.190840,-0.221335,12.22,1.21,...,20.98,0.21,24.06,29.02,22.59,31.35,3.07,12.77,102377008,Agricultura
3,ALSO3,24.04,25.57,2.31,5.58,4.02,4.582437,0.063644,21.84,0.88,...,172.55,-0.12,22.54,24.04,15.15,25.47,34.38,4.57,573936909,Exploracao de Imoveis
4,ALUP11,29.52,24.49,5.85,2.50,11.52,9.796000,-0.170393,10.19,1.17,...,23.99,1.22,28.53,29.64,22.35,30.11,16.86,3.47,914275704,Energia Eletrica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,VLID3,18.29,15.79,3.48,2.00,9.23,7.895000,-0.136687,12.68,1.17,...,8.63,0.35,18.29,19.16,7.84,19.16,92.53,-1.67,81836375,Servicos Diversos
167,VULC3,21.65,7.70,2.98,2.17,9.32,3.548387,-0.644342,28.24,2.63,...,19.83,0.05,20.27,21.94,10.62,21.94,56.88,5.82,245916244,Calcados
168,WIZC3,6.26,2.59,7.20,0.66,9.41,3.924242,-0.586262,25.32,2.38,...,10.49,0.10,6.13,6.46,4.91,8.41,-21.75,0.32,159907282,Corretoras de Seguros
169,WLMM3,25.00,17.94,3.14,3.11,8.08,5.768489,-0.282400,17.35,1.40,...,5.11,0.00,23.11,25.17,19.22,25.17,30.07,-0.68,36414670,Material de Transporte


In [7]:
value_invest = investing[['TICKERS', 'PRECO', 'MIN_MES', 'MAX_MES', 'VAL_MES', 'MIN_52S', 'MAX_52S',  'VAL_12M', 
                       ' VPA', 'DY', ' LPA', 'P/L' ,'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 'P/VP', 'EV/EBIT', 
                       'LIQ.CORR', 'MARG.LIQ', 'DIV.LIQ/PL', 'QTDEPAPEL', 'SEGMENTO']]

value_invest

,TICKERS,PRECO,MIN_MES,MAX_MES,VAL_MES,MIN_52S,MAX_52S,VAL_12M,VPA,DY,...,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL,QTDEPAPEL,SEGMENTO
0,ABCB4,19.40,18.35,19.40,3.91,15.27,21.24,3.58,23.98,8.43,...,6.890805,0.236082,14.50,0.78,4.25,1.50,19.35,0.00,226090118,Bancos
1,AFLT3,8.60,8.60,9.01,-3.37,8.07,10.50,-6.01,4.30,2.95,...,6.935484,-0.500000,14.46,2.07,14.50,4.34,48.96,-0.07,63084700,Energia Eletrica
2,AGRO3,27.56,24.06,29.02,12.77,22.59,31.35,3.07,21.46,12.49,...,8.190840,-0.221335,12.22,1.21,11.44,2.14,20.98,0.21,102377008,Agricultura
3,ALSO3,24.04,22.54,24.04,4.57,15.15,25.47,34.38,25.57,2.31,...,4.582437,0.063644,21.84,0.88,2.04,1.87,172.55,-0.12,573936909,Exploracao de Imoveis
4,ALUP11,29.52,28.53,29.64,3.47,22.35,30.11,16.86,24.49,5.85,...,9.796000,-0.170393,10.19,1.17,7.76,2.44,23.99,1.22,914275704,Energia Eletrica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,VLID3,18.29,18.29,19.16,-1.67,7.84,19.16,92.53,15.79,3.48,...,7.895000,-0.136687,12.68,1.17,5.21,2.03,8.63,0.35,81836375,Servicos Diversos
167,VULC3,21.65,20.27,21.94,5.82,10.62,21.94,56.88,7.70,2.98,...,3.548387,-0.644342,28.24,2.63,10.27,3.71,19.83,0.05,245916244,Calcados
168,WIZC3,6.26,6.13,6.46,0.32,4.91,8.41,-21.75,2.59,7.20,...,3.924242,-0.586262,25.32,2.38,2.72,0.84,10.49,0.10,159907282,Corretoras de Seguros
169,WLMM3,25.00,23.11,25.17,-0.68,19.22,25.17,30.07,17.94,3.14,...,5.768489,-0.282400,17.35,1.40,7.22,2.48,5.11,0.00,36414670,Material de Transporte
